In [3]:
import os
import mido
import json
import openai
from datetime import datetime
from dotenv import load_dotenv
from midi2audio import FluidSynth

load_dotenv()

True

In [12]:
def generate_music_from_text(text):
    openai.organization = os.getenv('OPENAI_ORGANIZATION')
    openai.api_key =os.getenv('OPENAI_API_KEY')
    # https://beta.openai.com/docs/api-reference/completions/create
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        user="experimental-notebook",
        messages=[
            {
                "content": "I need your support generating unique and original music based on the user input. Make sure to produce unique and original music inpsired by the user input",
                "role": "user",
            },
            {
                "content": "This is an example JSON file showcasing the format of the output I expect from you.",
                "role": "user",
            },
            {
                "content": """
              {
                "tempo": 80,
                "time_signature": "4/4",
                "tracks": [
                  {
                    "instrument": "Piano",
                    "notes": "A5-1.5, D5-0.5, F#5-1.5, D5-0.5, A5-1.5, D5-0.5, F#5-1.5, D5-0.5, A5-1.5, D5-0.5, F#5-1.5, D5-0.5, A5-1.5, D5-0.5, F#5-1.5, D5-0.5, A5-1.5, D5-0.5, F#5-1.5, D5-0.5, A5-1.5, D5-0.5, F#5-1.5, D5-0.5, A5-1.5, D5-0.5, F#5-1.5, D5-0.5",
                    "channel": 0
                  },
                  {
                    "instrument": "Strings",
                    "notes": "A3-1.5, D3-0.5, F#3-1.5, D3-0.5, A3-1.5, D3-0.5, F#3-1.5, D3-0.5, A3-1.5, D3-0.5, F#3-1.5, D3-0.5, A3-1.5, D3-0.5, F#3-1.5, D3-0.5, A3-1.5, D3-0.5, F#3-1.5, D3-0.5, A3-1.5, D3-0.5, F#3-1.5, D3-0.5, A3-1.5, D3-0.5, F#3-1.5, D3-0.5",
                    "channel": 1
                  },
                  {
                    "instrument": "Synth",
                    "notes": "A5-1.5, D6-0.5, F#5-1.5, D6-0.5, A5-1.5, D6-0.5, F#5-1.5, D6-0.5, A5-1.5, D6-0.5, F#5-1.5, D6-0.5, A5-1.5, D6-0.5, F#5-1.5, D6-0.5, A5-1.5, D6-0.5, F#5-1.5, D6-0.5, A5-1.5, D6-0.5, F#5-1.5, D6-0.5, A5-1.5, D6-0.5, F#5-1.5, D6-0.5",
                    "channel": 2
                  },
                  {
                    "instrument": "Drums",
                    "notes": "C2-1.0, C2-0.5, C2-0.5, C2-0.5, C2-0.5, C2-0.5, C2-0.5, C2-0.5, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, C2-0.25, G3-2.0, G3-2.0, G3-2.0, G3-2.0"
                  }
                ]
              }
              """,
                "role": "user",
            },
            {
                "content": "I already have a function that will take your output and convert into proper MIDI files + wav files. So no need to worry about that. I just need you to generate the JSON file. Just return the JSON file. No questions. No additional information. No comments. Only the JSON file. Your first letter is { and the last letter is }",
                "role": "user",
            },
            {
                "content": text,
                "role": "user",
            },
        ],
    )
    message = response["choices"][0]["message"]['content']
    print(message)
    return json.loads(message)

def parse_note(pitch):
    """Convert a pitch name to a MIDI note number."""
    # Define a dictionary of note names and their offsets from C
    note_names = {
        "C": 0,
        "C#": 1,
        "Db": 1,
        "D": 2,
        "D#": 3,
        "Eb": 3,
        "E": 4,
        "F": 5,
        "F#": 6,
        "Gb": 6,
        "G": 7,
        "G#": 8,
        "Ab": 8,
        "A": 9,
        "A#": 10,
        "Bb": 10,
        "B": 11
    }

    # Split the pitch name into note name and octave number
    note_name = pitch[:-1]
    octave = int(pitch[-1])

    # Calculate the MIDI note number based on the formula:
    # note_number = (octave + 1) * 12 + note_names[note_name]
    note_number = (octave + 1) * 12 + note_names[note_name]

    # Return the MIDI note number
    return note_number

def get_channel(instrument):
    instrument_to_channel = {
        "Piano": 0,
        "Strings": 1,
        "Guitar": 2,
        "Bass": 11,
        "Drums": 9,
        "Synth": 2,
        "Orchestra": 3
    }

    return instrument_to_channel[instrument]

In [16]:
file_name = 'automated'
score = generate_music_from_text('Combination of Experience by Einaudi and River Flows in You by Yiruma. Just use the Piano. Make it fast. Make it interesting. Make it short. I want to feel something.')

{
	"tempo": 160,
	"time_signature": "4/4",
	"tracks": [{
		"instrument": "Piano",
		"notes": "A4-1.0, A5-1.5, F#5-0.5, D5-0.5, E5-1.0, E6-1.5, C#6-0.5, A5-0.5, D5-1.0, D6-1.5, B5-0.5, G5-0.5, A4-1.0, A5-1.5, F#5-0.5, D5-0.5, E5-1.0, E6-1.5, C#6-0.5, A5-0.5, D5-1.0, D6-1.5, B5-0.5, G5-0.5, C#5-1.0, C#6-1.5, A5-0.5, F#5-0.5, G5-1.0, G6-1.5, E6-0.5, C#6-0.5, F#5-1.0, F#6-1.5, D6-0.5, B5-0.5, C#5-1.0, C#6-1.5, A5-0.5, F#5-0.5, G5-1.0, G6-1.5, E6-0.5, C#6-0.5, F#5-1.0, F#6-1.5, D6-0.5, B5-0.5",
		"channel": 0
	}]
}


In [18]:
# Create a MIDI file object
mid = mido.MidiFile()

# Create a track for each track in the score
for track in score["tracks"]:
    # Add a track name meta message
    mid_track = mido.MidiTrack()
    mid_track.append(mido.MetaMessage("track_name", name=track["instrument"]))

    # Set the tempo meta message based on the score tempo
    mid_track.append(mido.MetaMessage("set_tempo", tempo=mido.bpm2tempo(score["tempo"])))

    # Set the time signature meta message based on the score time signature
    num, den = map(int, score["time_signature"].split("/"))
    mid_track.append(mido.MetaMessage("time_signature", numerator=num, denominator=den))

    # Add note messages for each note in the track
    for note in track["notes"].split(', '):
        # Extract pitch and duration from the note string
        pitch, duration = note.split("-")
        pitch = parse_note(pitch.strip())
        duration = int(float(duration.strip()) * mid.ticks_per_beat * (score["tempo"] / 60))

        # Add note on and note off messages with velocity 64 and time 0
        mid_track.append(mido.Message("note_on",channel=get_channel(track['instrument']), note=pitch, velocity=64, time=0))
        mid_track.append(mido.Message("note_off",channel=get_channel(track['instrument']), note=pitch, velocity=64, time=duration))

    # Add end of track meta message
    mid_track.append(mido.MetaMessage("end_of_track"))

    # Append the track to the MIDI file object
    mid.tracks.append(mid_track)


# Create output directory with all relevant resources
output_directory = f"./output/{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
os.mkdir(output_directory)
json.dump(score, open(f"{output_directory}/{file_name}.json", "w"), indent=4)
# Save the MIDI file object as "labyrinth_timestamp.mid"
mid.save(f"{output_directory}/{file_name}.mid")
# Create a FluidSynth object with a sound font file
# fs = FluidSynth("Essential Keys-sforzando-v9.6.sf2")
fs = FluidSynth("FluidR3_GM.sf2")
# Convert the MIDI file to a WAV file
fs.midi_to_audio(f"{output_directory}/{file_name}.mid", f"{output_directory}/{file_name}.wav")


FluidSynth runtime version 2.3.1
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file './output/2023_03_05_11_27_54/automated.wav'..
